# Determining Feature Importances

This notebook is for exploring feature importance within a black box LSTM. I'll be starting with Permuation Importances and Shapley values.

### Building the Dataset

In [1]:
%cd ..
%cd .. 
%cd data

/Users/azel/github/Data-Science/jupyter_notebooks
/Users/azel/github/Data-Science
/Users/azel/github/Data-Science/data


In [2]:
from fin_data import DailyTimeSeries
import numpy as np
import pandas as pd
import os, sys
import matplotlib.pyplot as plt

    
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout


pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [14]:
def tech_company_data(ticker):
    """
    Containes Tech Index
    """    
    dts = DailyTimeSeries(ticker)
    df = dts.initiate()
    with HiddenPrints():
        df = dts.add_securities(['XLK', 'vix', 'SPX'], primary_df=df)
        df = dts.add_technicals(['SMA', 'EMA', 'MACD', 'STOCH', 
                                   'RSI', 'ADX', 'CCI', 'BBANDS', 
                                   'AD', 'OBV'], 
                                  primary_df=df)
        df = dts.add_macro(primary_df=df, 
                             indices=['trade_index', 'longterm_rates'])
    
    return df

In [15]:
df = tech_company_data('AAPL')

################################################################### 
 Ticker:  AAPL 
 Last Refreshed:  2019-09-04 
 Data Retrieved:  Daily Time Series with Splits and Dividend Events 
 ###################################################################


In [16]:
df.tail()

,AAPL_open,AAPL_high,AAPL_low,AAPL_close,AAPL_adjusted_close,AAPL_volume,AAPL_dividend_amount,XLK_open,XLK_high,XLK_low,XLK_close,XLK_adjusted_close,XLK_volume,XLK_dividend_amount,XLK_split_coefficient,vix_open,vix_high,vix_low,vix_close,vix_adjusted_close,vix_volume,vix_dividend_amount,vix_split_coefficient,SPX_open,SPX_high,SPX_low,SPX_close,SPX_adjusted_close,SPX_volume,SPX_dividend_amount,SPX_split_coefficient,AAPL_SMA,AAPL_EMA,AAPL_MACD_Signal,AAPL_MACD_Hist,AAPL_MACD,AAPL_SlowD,AAPL_SlowK,AAPL_RSI,AAPL_ADX,AAPL_CCI,AAPL_Real Upper Band,AAPL_Real Middle Band,AAPL_Real Lower Band,AAPL_Chaikin A/D,AAPL_OBV,trade_value,10 Yrs Rates,20-Yr Maturity Rate
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-08-26,205.86,207.19,205.06,206.49,206.49,26043600.0,0.0,78.12,78.27,77.59,78.24,78.24,10358500.0,0.0,1.0,20.34,21.33,19.06,19.32,19.32,0.0,0.0,1.0,2866.7000,2879.2700,2856.0000,2878.3799,2878.3799,2.857600e+09,0.0,1.0,205.1725,205.9112,1.3034,0.1175,1.4209,63.6566,44.7905,52.1186,15.1664,10.7310,215.9278,205.1725,194.4173,1.848775e+10,2.256327e+10,130.4981,1.94,1.84
2019-08-27,207.86,208.55,203.53,204.16,204.16,25873300.0,0.0,78.75,78.89,77.67,78.12,78.12,10033000.0,0.0,1.0,20.18,21.04,18.49,20.31,20.31,0.0,0.0,1.0,2893.1399,2898.7900,2860.5901,2869.1599,2869.1599,3.533630e+09,0.0,1.0,204.9415,205.7444,1.2649,-0.1542,1.1107,44.1370,25.5208,50.1842,14.4296,1.3501,215.5747,204.9415,194.3083,1.846837e+10,2.253740e+10,130.4981,1.86,1.77
2019-08-28,204.10,205.72,203.32,205.53,205.53,15938800.0,0.0,77.70,78.32,77.14,78.19,78.19,7139500.0,0.0,1.0,20.55,21.64,19.10,19.35,19.35,0.0,0.0,1.0,2861.2800,2890.0300,2853.0500,2887.9399,2887.9399,3.097420e+09,0.0,1.0,204.5660,205.7240,1.2047,-0.2405,0.9642,34.3333,32.6885,51.3028,13.7513,0.8910,214.5386,204.5660,194.5934,1.848179e+10,2.255334e+10,131.0308,1.85,1.76
2019-08-29,208.50,209.32,206.66,209.01,209.01,20990500.0,0.0,79.25,79.79,78.97,79.55,79.55,8120600.0,0.0,1.0,19.02,19.20,17.60,17.88,17.88,0.0,0.0,1.0,2910.3701,2930.5000,2905.6699,2924.5801,2924.5801,3.176190e+09,0.0,1.0,204.5950,206.0369,1.1870,-0.0709,1.1161,33.8149,43.2353,54.0612,13.3885,61.9063,214.6156,204.5950,194.5744,1.849789e+10,2.257433e+10,131.0308,1.87,1.78
2019-08-30,210.16,210.45,207.20,208.74,208.74,21143400.0,0.0,80.07,80.17,78.96,79.53,79.53,10075400.0,0.0,1.0,17.94,19.18,17.09,18.98,18.98,0.0,0.0,1.0,2937.0901,2940.4299,2913.3201,2926.4600,2926.4600,3.008450e+09,0.0,1.0,204.8310,206.2944,1.1898,0.0110,1.2008,45.5536,60.7370,53.8122,13.1538,63.6379,215.0074,204.8310,194.6546,1.849678e+10,2.255318e+10,131.0308,1.87,1.78
